In [1]:
!pip install beartype moviepy zsvision ruptures --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.0.post2 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.


In [2]:
from beartype import beartype
from moviepy.editor import *
from pathlib import Path
# from pytube import YouTube, Playlist
from scipy.io import loadmat
from tqdm import tqdm
from tqdm.notebook import tqdm
from zsvision.zs_utils import BlockTimer
import argparse
import cv2
# import ffmpeg
import math
import matplotlib.pyplot as plt
# import models
import numpy as np
import os
import pandas as pd
import pickle as pkl
import scipy.special
import shutil
import sys
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
def torch_to_list(torch_tensor):
    return torch_tensor.cpu().numpy().tolist()

def save_pred(preds, checkpoint="checkpoint", filename="preds_valid.mat"):
    preds = to_numpy(preds)
    os.makedirs(checkpoint, exist_ok=True)
    filepath = os.path.join(checkpoint, filename)
    mdict = {"preds": preds}
    print(f"Saving to {filepath}")
    scipy.io.savemat(filepath, mdict=mdict, do_compression=False, format="4")

def to_torch(ndarray):
    if type(ndarray).__module__ == "numpy":
        return torch.from_numpy(ndarray)
    elif not torch.is_tensor(ndarray):
        raise ValueError(f"Cannot convert {type(ndarray)} to torch tensor")
    return ndarray

def to_numpy(tensor):
    if torch.is_tensor(tensor):
        return tensor.cpu().numpy()
    elif type(tensor).__module__ != "numpy":
        raise ValueError(f"Cannot convert {type(tensor)} to numpy array")
    return tensor

def im_to_numpy(img):
    img = to_numpy(img)
    img = np.transpose(img, (1, 2, 0))  # H*W*C
    return img


def im_to_torch(img):
    img = np.transpose(img, (2, 0, 1))  # C*H*W
    img = to_torch(img).float()
    if img.max() > 1:
        img /= 255
    return img


def resize_generic(img, oheight, owidth, interp="bilinear", is_flow=False):
    """
    Args
    inp: numpy array: RGB image (H, W, 3) | video with 3*nframes (H, W, 3*nframes)
          |  single channel image (H, W, 1) | -- not supported:  video with (nframes, 3, H, W)
    """

    # resized_image = cv2.resize(image, (100, 50))
    ht, wd, chn = img.shape[0], img.shape[1], img.shape[2]
    if chn == 1:
        resized_img = scipy.misc.imresize(
            img.squeeze(), [oheight, owidth], interp=interp, mode="F"
        ).reshape((oheight, owidth, chn))
    elif chn == 3:
        # resized_img = scipy.misc.imresize(img, [oheight, owidth], interp=interp)  # mode='F' gives an error for 3 channels
        resized_img = cv2.resize(img, (owidth, oheight))  # inverted compared to scipy
    elif chn == 2:
        # assert(is_flow)
        resized_img = np.zeros((oheight, owidth, chn), dtype=img.dtype)
        for t in range(chn):
            # resized_img[:, :, t] = scipy.misc.imresize(img[:, :, t], [oheight, owidth], interp=interp)
            # resized_img[:, :, t] = scipy.misc.imresize(img[:, :, t], [oheight, owidth], interp=interp, mode='F')
            # resized_img[:, :, t] = np.array(Image.fromarray(img[:, :, t]).resize([oheight, owidth]))
            resized_img[:, :, t] = scipy.ndimage.interpolation.zoom(
                img[:, :, t], [oheight, owidth]
            )
    else:
        in_chn = 3
        # Workaround, would be better to pass #frames
        if chn == 16:
            in_chn = 1
        if chn == 32:
            in_chn = 2
        nframes = int(chn / in_chn)
        img = img.reshape(img.shape[0], img.shape[1], in_chn, nframes)
        resized_img = np.zeros((oheight, owidth, in_chn, nframes), dtype=img.dtype)
        for t in range(nframes):
            frame = img[:, :, :, t]  # img[:, :, t*3:t*3+3]
            frame = cv2.resize(frame, (owidth, oheight)).reshape(
                oheight, owidth, in_chn
            )
            # frame = scipy.misc.imresize(frame, [oheight, owidth], interp=interp)
            resized_img[:, :, :, t] = frame
        resized_img = resized_img.reshape(
            resized_img.shape[0], resized_img.shape[1], chn
        )

    if is_flow:
        # print(oheight / ht)
        # print(owidth / wd)
        resized_img = resized_img * oheight / ht
    return resized_img


def color_normalize(x, mean, std):
    """Normalize a tensor of images by subtracting (resp. dividing) by the mean (resp.
    std. deviation) statistics of a dataset in RGB space.
    """
    if x.dim() in {3, 4}:
        if x.size(0) == 1:
            x = x.repeat(3, 1, 1)
        assert x.size(0) == 3, "For single video format, expected RGB along first dim"
        for t, m, s in zip(x, mean, std):
            t.sub_(m)
            t.div_(s)
    elif x.dim() == 5:
        assert (
            x.shape[1] == 3
        ), "For batched video format, expected RGB along second dim"
        x[:, 0].sub_(mean[0]).div_(std[0])
        x[:, 1].sub_(mean[1]).div_(std[1])
        x[:, 2].sub_(mean[2]).div_(std[2])
    return x

def get_labels_start_end_time(frame_wise_labels, bg_class=["Sign"]):
    """get list of start and end times of each interval/ segment.

    Args:
        frame_wise_labels: list of framewise labels/ predictions.
        bg_class: list of all classes in frame_wise_labels which should be ignored

    Returns:
        labels: list of labels of the segments
        starts: list of start times of the segments
        ends: list of end times of the segments
    """
    labels = []
    starts = []
    ends = []
    last_label = frame_wise_labels[0]
    # print(frame_wise_labels, bg_class)
    if frame_wise_labels[0] not in bg_class:
        labels.append(frame_wise_labels[0])
        starts.append(0)
    for i in range(len(frame_wise_labels)):
        if frame_wise_labels[i] != last_label:
            if frame_wise_labels[i] not in bg_class:
                labels.append(frame_wise_labels[i])
                starts.append(i)
            if last_label not in bg_class:
                ends.append(i)
            last_label = frame_wise_labels[i]
    if last_label not in bg_class:
        ends.append(i + 1)
    return labels, starts, ends

def generate_vtt_file(all_preds, logits, save_path):
    vtt = WebVTT()
    predictions = all_preds

    labels, starts, ends = get_labels_start_end_time(predictions, [1])

    # smaller boundaries
    for ix in range(len(labels)):
        if ix == len(labels)-1:
            break
        diff = starts[ix+1]-ends[ix]
        starts[ix+1] -= floor(diff/2)
        ends[ix] += floor(diff/2)
    
    if logits is not None:
        # load i3d classes
        i3d_scores = logits
        with open('data/info/bslcp/info.pkl', 'rb') as f:
            info_data = pickle.load(f)

    # for start, end in zip(starts, ends):
    for start, end in zip(starts, ends):

        if logits is not None:
            i3d_score = np.sum(np.asarray(i3d_scores)[start:end], axis=0)
            ind = np.argpartition(i3d_score, -10)[-10:]       
            ind = ind[np.argsort(-i3d_score[ind])]
            classes = [info_data['words'][ix] for ix in ind]

            class_str = ','.join(classes)
        else:
            class_str = ''

        start = (start + 8) / 25
        end = (end + 8) / 25

        start_dt = datetime.timedelta(seconds=start)
        start_str = str(start_dt)
        if '.' not in start_str:
            start_str = f'{start_str}.000000'

        end_dt = datetime.timedelta(seconds=end)
        end_str = str(end_dt)
        if '.' not in end_str:
            end_str = f'{end_str}.000000'
        # creating a caption with a list of lines
        caption = Caption(
            start_str,
            end_str,
            [class_str]
        )

        # adding a caption
        vtt.captions.append(caption)


    # save to a different file
    vtt.save(f'{save_path}/demo.vtt')

In [6]:
def load_rgb_video(video_path: Path, fps: int) -> torch.Tensor:
    """
    Load frames of a video using cv2.
    """
    cap = cv2.VideoCapture(str(video_path))
    cap_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    cap_fps = cap.get(cv2.CAP_PROP_FPS)

    # cv2 won't be able to change frame rates for all encodings, so we use ffmpeg
    if cap_fps != fps:
        tmp_video_path = f"{video_path}.tmp.{video_path.suffix}"
        shutil.move(video_path, tmp_video_path)
        cmd = (f"ffmpeg -i {tmp_video_path} -pix_fmt yuv420p "
               f"-filter:v fps=fps={fps} {video_path}")
        print(f"Generating new copy of video with frame rate {fps}")
        os.system(cmd)
        Path(tmp_video_path).unlink()
        cap = cv2.VideoCapture(str(video_path))
        cap_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        cap_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        cap_fps = cap.get(cv2.CAP_PROP_FPS)
        assert cap_fps == fps, f"ffmpeg failed to produce a video at {fps}"

    f = 0
    rgb = []
    while True:
        # frame: BGR, (h, w, 3), dtype=uint8 0..255
        ret, frame = cap.read()
        if not ret:
            break
        # BGR (OpenCV) to RGB (Torch)
        frame = frame[:, :, [2, 1, 0]]
        rgb_t = im_to_torch(frame)
        rgb.append(rgb_t)
        f += 1
    cap.release()
    # (nframes, 3, cap_height, cap_width) => (3, nframes, cap_height, cap_width)
    rgb = torch.stack(rgb).permute(1, 0, 2, 3)
    print(f"Loaded video {video_path} with {f} frames [{cap_height}hx{cap_width}w] res. "
          f"at {cap_fps}")
    return rgb

In [7]:
def prepare_input(
    rgb: torch.Tensor,
    resize_res: int = 256,
    inp_res: int = 224,
    mean: torch.Tensor = 0.5 * torch.ones(3), std=1.0 * torch.ones(3),
):
    """
    Process the video:
    1) Resize to [resize_res x resize_res]
    2) Center crop with [inp_res x inp_res]
    3) Color normalize using mean/std
    """
    iC, iF, iH, iW = rgb.shape
    rgb_resized = np.zeros((iF, resize_res, resize_res, iC))
    for t in range(iF):
        tmp = rgb[:, t, :, :]
        tmp = resize_generic(
            im_to_numpy(tmp), resize_res, resize_res, interp="bilinear", is_flow=False
        )
        rgb_resized[t] = tmp
    rgb = np.transpose(rgb_resized, (3, 0, 1, 2))
    # Center crop coords
    ulx = int((resize_res - inp_res) / 2)
    uly = int((resize_res - inp_res) / 2)
    # Crop 256x256
    rgb = rgb[:, :, uly : uly + inp_res, ulx : ulx + inp_res]
    rgb = to_torch(rgb).float()
    assert rgb.max() <= 1
    rgb = color_normalize(rgb, mean, std)
    return rgb

import torch
import torchvision.transforms as transforms
from typing import Tuple

In [8]:
def sliding_windows(
        rgb: torch.Tensor,
        num_in_frames: int,
        stride: int,
) -> tuple:
    """
    Return sliding windows and corresponding (middle) timestamp
    """
    C, nFrames, H, W = rgb.shape
    # If needed, pad to the minimum clip length
    if nFrames < num_in_frames:
        rgb_ = torch.zeros(C, num_in_frames, H, W)
        rgb_[:, :nFrames] = rgb
        rgb_[:, nFrames:] = rgb[:, -1].unsqueeze(1)
        rgb = rgb_
        nFrames = rgb.shape[1]

    num_clips = math.ceil((nFrames - num_in_frames) / stride) + 1
    plural = ""
    if num_clips > 1:
        plural = "s"
    print(f"{num_clips} clip{plural} resulted from sliding window processing.")

    rgb_slided = torch.zeros(num_clips, 3, num_in_frames, H, W)
    t_mid = []
    # For each clip
    for j in range(num_clips):
        # Check if num_clips becomes 0
        actual_clip_length = min(num_in_frames, nFrames - j * stride)
        if actual_clip_length == num_in_frames:
            t_beg = j * stride
        else:
            t_beg = nFrames - num_in_frames
        t_mid.append(t_beg + num_in_frames / 2)
        rgb_slided[j] = rgb[:, t_beg : t_beg + num_in_frames, :, :]
    return rgb_slided, np.array(t_mid)

In [9]:
class MaxPool3dSamePadding(nn.MaxPool3d):
    def compute_pad(self, dim, s):
        if s % self.stride[dim] == 0:
            return max(self.kernel_size[dim] - self.stride[dim], 0)
        else:
            return max(self.kernel_size[dim] - (s % self.stride[dim]), 0)

    def forward(self, x):
        # compute 'same' padding
        (batch, channel, t, h, w) = x.size()
        # print t,h,ms.shaw
        # out_t = np.ceil(float(t) / float(self.stride[0]))
        # out_h = np.ceil(float(h) / float(self.stride[1]))
        # out_w = np.ceil(float(w) / float(self.stride[2]))
        # print out_t, out_h, out_w
        pad_t = self.compute_pad(0, t)
        pad_h = self.compute_pad(1, h)
        pad_w = self.compute_pad(2, w)
        # print pad_t, pad_h, pad_w

        pad_t_f = pad_t // 2
        pad_t_b = pad_t - pad_t_f
        pad_h_f = pad_h // 2
        pad_h_b = pad_h - pad_h_f
        pad_w_f = pad_w // 2
        pad_w_b = pad_w - pad_w_f

        pad = (pad_w_f, pad_w_b, pad_h_f, pad_h_b, pad_t_f, pad_t_b)
        # print x.size()
        # print pad
        x = F.pad(x, pad)
        return super(MaxPool3dSamePadding, self).forward(x)

In [10]:
class Unit3D(nn.Module):
    def __init__(
        self,
        in_channels,
        output_channels,
        kernel_shape=(1, 1, 1),
        stride=(1, 1, 1),
        padding=0,
        activation_fn=F.relu,
        use_batch_norm=True,
        use_bias=False,
        name="unit_3d",
        num_domains=1,
    ):

        """Initializes Unit3D module."""
        super(Unit3D, self).__init__()

        self._output_channels = output_channels
        self._kernel_shape = kernel_shape
        self._stride = stride
        self._use_batch_norm = use_batch_norm
        self._num_domains = num_domains
        self._activation_fn = activation_fn
        self._use_bias = use_bias
        self.name = name
        self.padding = padding

        self.conv3d = nn.Conv3d(
            in_channels=in_channels,
            out_channels=self._output_channels,
            kernel_size=self._kernel_shape,
            stride=self._stride,
            padding=0,  # we always want padding to be 0 here. We will dynamically pad based on input size in forward function
            bias=self._use_bias,
        )

        if self._use_batch_norm:
            if self._num_domains == 1:
                self.bn = nn.BatchNorm3d(self._output_channels, eps=0.001, momentum=0.01)
            else:
                self.bn = DomainSpecificBatchNorm3d(
                    self._output_channels, self._num_domains, eps=0.001, momentum=0.01
                )

    def compute_pad(self, dim, s):
        if s % self._stride[dim] == 0:
            return max(self._kernel_shape[dim] - self._stride[dim], 0)
        else:
            return max(self._kernel_shape[dim] - (s % self._stride[dim]), 0)

    def forward(self, x):
        # compute 'same' padding
        (batch, channel, t, h, w) = x.size()
        # print t,h,w
        # out_t = np.ceil(float(t) / float(self._stride[0]))
        # out_h = np.ceil(float(h) / float(self._stride[1]))
        # out_w = np.ceil(float(w) / float(self._stride[2]))
        # print out_t, out_h, out_w
        pad_t = self.compute_pad(0, t)
        pad_h = self.compute_pad(1, h)
        pad_w = self.compute_pad(2, w)
        # print pad_t, pad_h, pad_w

        pad_t_f = pad_t // 2
        pad_t_b = pad_t - pad_t_f
        pad_h_f = pad_h // 2
        pad_h_b = pad_h - pad_h_f
        pad_w_f = pad_w // 2
        pad_w_b = pad_w - pad_w_f

        pad = (pad_w_f, pad_w_b, pad_h_f, pad_h_b, pad_t_f, pad_t_b)
        # print x.size()
        # print pad
        x = F.pad(x, pad)
        # print x.size()

        x = self.conv3d(x)
        if self._use_batch_norm:
            x = self.bn(x)
        if self._activation_fn is not None:
            x = self._activation_fn(x)
        return x

In [11]:
class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels, name, num_domains=1):
        super(InceptionModule, self).__init__()

        self.b0 = Unit3D(
            in_channels=in_channels,
            output_channels=out_channels[0],
            kernel_shape=[1, 1, 1],
            padding=0,
            name=name + "/Branch_0/Conv3d_0a_1x1",
        )
        self.b1a = Unit3D(
            in_channels=in_channels,
            output_channels=out_channels[1],
            kernel_shape=[1, 1, 1],
            padding=0,
            name=name + "/Branch_1/Conv3d_0a_1x1",
        )
        self.b1b = Unit3D(
            in_channels=out_channels[1],
            output_channels=out_channels[2],
            kernel_shape=[3, 3, 3],
            name=name + "/Branch_1/Conv3d_0b_3x3",
        )
        self.b2a = Unit3D(
            in_channels=in_channels,
            output_channels=out_channels[3],
            kernel_shape=[1, 1, 1],
            padding=0,
            name=name + "/Branch_2/Conv3d_0a_1x1",
        )
        self.b2b = Unit3D(
            in_channels=out_channels[3],
            output_channels=out_channels[4],
            kernel_shape=[3, 3, 3],
            name=name + "/Branch_2/Conv3d_0b_3x3",
        )
        self.b3a = MaxPool3dSamePadding(
            kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=0
        )
        self.b3b = Unit3D(
            in_channels=in_channels,
            output_channels=out_channels[5],
            kernel_shape=[1, 1, 1],
            padding=0,
            name=name + "/Branch_3/Conv3d_0b_1x1",
        )
        self.name = name

    def forward(self, x):
        b0 = self.b0(x)
        b1 = self.b1b(self.b1a(x))
        b2 = self.b2b(self.b2a(x))
        b3 = self.b3b(self.b3a(x))
        return torch.cat([b0, b1, b2, b3], dim=1)

In [12]:
class InceptionI3d(nn.Module):
    """Inception-v1 I3D architecture.
    The model is introduced in:
        Quo Vadis, Action Recognition? A New Model and the Kinetics Dataset
        Joao Carreira, Andrew Zisserman
        https://arxiv.org/pdf/1705.07750v1.pdf.
    See also the Inception architecture, introduced in:
        Going deeper with convolutions
        Christian Szegedy, Wei Liu, Yangqing Jia, Pierre Sermanet, Scott Reed,
        Dragomir Anguelov, Dumitru Erhan, Vincent Vanhoucke, Andrew Rabinovich.
        http://arxiv.org/pdf/1409.4842v1.pdf.
    """

    # Endpoints of the model in order. During construction, all the endpoints up
    # to a designated `final_endpoint` are returned in a dictionary as the
    # second return value.
    VALID_ENDPOINTS = (
        "Conv3d_1a_7x7",
        "MaxPool3d_2a_3x3",
        "Conv3d_2b_1x1",
        "Conv3d_2c_3x3",
        "MaxPool3d_3a_3x3",
        "Mixed_3b",
        "Mixed_3c",
        "MaxPool3d_4a_3x3",
        "Mixed_4b",
        "Mixed_4c",
        "Mixed_4d",
        "Mixed_4e",
        "Mixed_4f",
        "MaxPool3d_5a_2x2",
        "Mixed_5b",
        "Mixed_5c",
        "Logits",
        "Predictions",
    )

    def __init__(
        self,
        num_classes=400,
        spatiotemporal_squeeze=True,
        final_endpoint="Logits",
        name="inception_i3d",
        in_channels=3,
        dropout_keep_prob=0.5,
        num_in_frames=64,
        include_embds=False,
    ):
        """Initializes I3D model instance.
        Args:
          num_classes: The number of outputs in the logit layer (default 400, which
              matches the Kinetics dataset).
          spatiotemporal_squeeze: Whether to squeeze the 2 spatial and 1 temporal dimensions for the logits
              before returning (default True).
          final_endpoint: The model contains many possible endpoints.
              `final_endpoint` specifies the last endpoint for the model to be built
              up to. In addition to the output at `final_endpoint`, all the outputs
              at endpoints up to `final_endpoint` will also be returned, in a
              dictionary. `final_endpoint` must be one of
              InceptionI3d.VALID_ENDPOINTS (default 'Logits').
          in_channels: Number of input channels (default 3 for RGB).
          dropout_keep_prob: Dropout probability (default 0.5).
          name: A string (optional). The name of this module.
          num_in_frames: Number of input frames (default 64).
          include_embds: Whether to return embeddings (default False).
        Raises:
          ValueError: if `final_endpoint` is not recognized.
        """

        if final_endpoint not in self.VALID_ENDPOINTS:
            raise ValueError("Unknown final endpoint %s" % final_endpoint)

        super().__init__()
        self._num_classes = num_classes
        self._spatiotemporal_squeeze = spatiotemporal_squeeze
        self._final_endpoint = final_endpoint
        self.include_embds = include_embds
        self.logits = None

        if self._final_endpoint not in self.VALID_ENDPOINTS:
            raise ValueError("Unknown final endpoint %s" % self._final_endpoint)

        self.end_points = {}
        end_point = "Conv3d_1a_7x7"
        self.end_points[end_point] = Unit3D(
            in_channels=in_channels,
            output_channels=64,
            kernel_shape=[7, 7, 7],
            stride=(2, 2, 2),
            padding=(3, 3, 3),
            name=name + end_point,
        )
        if self._final_endpoint == end_point:
            return

        end_point = "MaxPool3d_2a_3x3"
        self.end_points[end_point] = MaxPool3dSamePadding(
            kernel_size=[1, 3, 3], stride=(1, 2, 2), padding=0
        )
        if self._final_endpoint == end_point:
            return

        end_point = "Conv3d_2b_1x1"
        self.end_points[end_point] = Unit3D(
            in_channels=64,
            output_channels=64,
            kernel_shape=[1, 1, 1],
            padding=0,
            name=name + end_point,
        )
        if self._final_endpoint == end_point:
            return

        end_point = "Conv3d_2c_3x3"
        self.end_points[end_point] = Unit3D(
            in_channels=64,
            output_channels=192,
            kernel_shape=[3, 3, 3],
            padding=1,
            name=name + end_point,
        )
        if self._final_endpoint == end_point:
            return

        end_point = "MaxPool3d_3a_3x3"
        self.end_points[end_point] = MaxPool3dSamePadding(
            kernel_size=[1, 3, 3], stride=(1, 2, 2), padding=0
        )
        if self._final_endpoint == end_point:
            return

        end_point = "Mixed_3b"
        self.end_points[end_point] = InceptionModule(
            192, [64, 96, 128, 16, 32, 32], name + end_point,
        )
        if self._final_endpoint == end_point:
            return

        end_point = "Mixed_3c"
        self.end_points[end_point] = InceptionModule(
            256, [128, 128, 192, 32, 96, 64], name + end_point,
        )
        if self._final_endpoint == end_point:
            return

        end_point = "MaxPool3d_4a_3x3"
        self.end_points[end_point] = MaxPool3dSamePadding(
            kernel_size=[3, 3, 3], stride=(2, 2, 2), padding=0
        )
        if self._final_endpoint == end_point:
            return

        end_point = "Mixed_4b"
        self.end_points[end_point] = InceptionModule(
            128 + 192 + 96 + 64, [192, 96, 208, 16, 48, 64], name + end_point,
        )
        if self._final_endpoint == end_point:
            return

        end_point = "Mixed_4c"
        self.end_points[end_point] = InceptionModule(
            192 + 208 + 48 + 64, [160, 112, 224, 24, 64, 64], name + end_point,
        )
        if self._final_endpoint == end_point:
            return

        end_point = "Mixed_4d"
        self.end_points[end_point] = InceptionModule(
            160 + 224 + 64 + 64, [128, 128, 256, 24, 64, 64], name + end_point,
        )
        if self._final_endpoint == end_point:
            return

        end_point = "Mixed_4e"
        self.end_points[end_point] = InceptionModule(
            128 + 256 + 64 + 64, [112, 144, 288, 32, 64, 64], name + end_point,
        )
        if self._final_endpoint == end_point:
            return

        end_point = "Mixed_4f"
        self.end_points[end_point] = InceptionModule(
            112 + 288 + 64 + 64, [256, 160, 320, 32, 128, 128], name + end_point,
        )
        if self._final_endpoint == end_point:
            return

        end_point = "MaxPool3d_5a_2x2"
        self.end_points[end_point] = MaxPool3dSamePadding(
            kernel_size=[2, 2, 2], stride=(2, 2, 2), padding=0
        )
        if self._final_endpoint == end_point:
            return

        end_point = "Mixed_5b"
        self.end_points[end_point] = InceptionModule(
            256 + 320 + 128 + 128, [256, 160, 320, 32, 128, 128], name + end_point,
        )
        if self._final_endpoint == end_point:
            return

        end_point = "Mixed_5c"
        self.end_points[end_point] = InceptionModule(
            256 + 320 + 128 + 128, [384, 192, 384, 48, 128, 128], name + end_point,
        )
        if self._final_endpoint == end_point:
            return

        end_point = "Logits"

        last_duration = int(math.ceil(num_in_frames / 8))  # 8
        last_size = 7  # int(math.ceil(sample_width / 32))  # this is for 224
        self.avgpool = nn.AvgPool3d((last_duration, last_size, last_size), stride=1)

        self.dropout = nn.Dropout(dropout_keep_prob)

        self.logits = Unit3D(
            in_channels=384 + 384 + 128 + 128,
            output_channels=self._num_classes,
            kernel_shape=[1, 1, 1],
            padding=0,
            activation_fn=None,
            use_batch_norm=False,
            use_bias=True,
            name="logits",
        )

        self.build()

    def replace_logits(self, num_classes):
        self._num_classes = num_classes
        self.logits = Unit3D(
            in_channels=384 + 384 + 128 + 128,
            output_channels=self._num_classes,
            kernel_shape=[1, 1, 1],
            padding=0,
            activation_fn=None,
            use_batch_norm=False,
            use_bias=True,
            name="logits",
        )

    def build(self):
        for k in self.end_points.keys():
            self.add_module(k, self.end_points[k])

    def forward(self, x):
        for end_point in self.VALID_ENDPOINTS:
            if end_point in self.end_points:
                x = self._modules[end_point](x)
        # [batch x featuredim x 1 x 1 x 1]
        embds = self.dropout(self.avgpool(x))

        # [batch x classes x 1 x 1 x 1]
        x = self.logits(embds)
        if self._spatiotemporal_squeeze:
            # [batch x classes]
            logits = x.squeeze(3).squeeze(3).squeeze(2)

        # logits [batch X classes]
        if self.include_embds:
            return {"logits": logits, "embds": embds}
        else:
            return {"logits": logits}

In [13]:
def load_i3d_model(
        i3d_checkpoint_path: Path,
        num_classes: int,
        num_in_frames: int) -> torch.nn.Module:
    
    """Load pre-trained I3D checkpoint, put in eval mode."""
    model = InceptionI3d(
        num_classes=num_classes,
        spatiotemporal_squeeze=True,
        final_endpoint="Logits",
        name="inception_i3d",
        in_channels=3,
        dropout_keep_prob=0.5,
        num_in_frames=num_in_frames,
        include_embds=True,
    )
    model = torch.nn.DataParallel(model).cuda()
    checkpoint = torch.load(i3d_checkpoint_path)
    model.load_state_dict(checkpoint["state_dict"])
    model.eval()
    return model

In [544]:
i3dmodel = load_i3d_model(
            i3d_checkpoint_path= Path("/home/jupyter/imported/models/i3d/i3d_kinetics_bslcp.pth.tar"),
            num_classes=981,
            num_in_frames= 16,
        )

In [14]:
def main_i3d(
    #starting_point: str = "feature",
    #i3d_checkpoint_path: Path = Path("/kaggle/input/sign-seg-models/models/i3d/i3d_kinetics_bsl1k_bslcp.pth.tar"),
    #mstcn_checkpoint_path: Path,
    video_path: Path,
    #feature_path: Path,
    save_path: Path,
    fps: int = 25,
    num_classes: int = 981,
    num_in_frames: int = 16,
    batch_size: int = 16,
    stride: int = 1,
    slowdown_factor: int = 1,
    save_features: bool = True,
    #save_segments: bool = True,
    viz: bool = False,
    generate_vtt: bool = False,
):
    
    with BlockTimer("Loading video frames"):
        rgb_orig = load_rgb_video(
            video_path=video_path,
            fps=fps,
        )
    # Prepare: resize/crop/normalize
    rgb_input = prepare_input(rgb_orig)
    # Sliding window
    rgb_slides, t_mid = sliding_windows(
        rgb=rgb_input,
        stride=stride,
        num_in_frames=num_in_frames,
    )
    # Number of windows/clips
    num_clips = rgb_slides.shape[0]
    # Group the clips into batches
    num_batches = math.ceil(num_clips / batch_size)
    all_features = torch.Tensor(num_clips, 1024)
    all_logits = torch.Tensor(num_clips, num_classes)
    for b in range(num_batches):
        inp = rgb_slides[b * batch_size : (b + 1) * batch_size]
        # Forward pass
        out = i3dmodel(inp)
        logits = out["logits"].data.cpu()
        all_features[b*batch_size:(b+1)*batch_size] = out["embds"].squeeze().data.cpu()
        all_logits[b*batch_size:(b+1)*batch_size] = logits.squeeze().data.cpu()

    if save_features:
        save_pred(
            all_features, checkpoint=save_path, filename=Path(str(video_path).split('/')[-1][:-4]+'.mat'),
        )
    
    return all_features, all_logits

In [546]:
# features, logits = main_i3d(Path('/home/jupyter/imported/yt_scrape/final_data/2022-April-video0.mp4'), '/home/jupyter/imported/features-i3d-mstcn')

In [547]:
os.path.isfile(os.path.join(save_path, str(video_path).split('/')[-1][:-4]+'.mat'))

True

In [549]:
for videos in tqdm(os.listdir('/home/jupyter/imported/yt_scrape/final_data')[:]):
    if videos[-4:] != '.mp4':
        continue
    video_path = Path(os.path.join(dataset_path,videos))
    save_path = Path('/home/jupyter/imported/features-i3d-mstcn')
    if os.path.isfile(os.path.join(save_path, str(video_path).split('/')[-1][:-4]+'.mat')):
        continue
    main_i3d(video_path, save_path)

  0%|          | 0/699 [00:00<?, ?it/s]

Loading video frames...Loaded video /home/jupyter/imported/yt_scrape/final_data/2021-November-video4.mp4 with 500 frames [720hx600w] res. at 25.0

 took 00h00m01s

485 clips resulted from sliding window processing.


  1%|          | 4/699 [00:09<28:42,  2.48s/it]

Saving to /home/jupyter/imported/features-i3d-mstcn/2021-November-video4.mat

Loading video frames...Loaded video /home/jupyter/imported/yt_scrape/final_data/2021-November-video17.mp4 with 500 frames [720hx600w] res. at 25.0

 took 00h00m01s

485 clips resulted from sliding window processing.


  2%|▏         | 15/699 [00:19<14:01,  1.23s/it]

...

Saving to /home/jupyter/imported/features-i3d-mstcn/2021-December-video7.mat

Loading video frames...Loaded video /home/jupyter/imported/yt_scrape/final_data/2021-October-video4.mp4 with 500 frames [720hx600w] res. at 25.0

 took 00h00m01s

485 clips resulted from sliding window processing.


 99%|█████████▉| 694/699 [43:09<00:23,  4.70s/it]

Saving to /home/jupyter/imported/features-i3d-mstcn/2021-October-video4.mat

Loading video frames...Loaded video /home/jupyter/imported/yt_scrape/final_data/2021-May-video26.mp4 with 500 frames [720hx600w] res. at 25.0

 took 00h00m01s

485 clips resulted from sliding window processing.


100%|██████████| 699/699 [43:19<00:00,  3.72s/it]

Saving to /home/jupyter/imported/features-i3d-mstcn/2021-May-video26.mat


In [15]:
import copy
import argparse
import pickle
from math import floor, ceil
import numpy as np
import cv2
import seaborn as sns
from pathlib import Path

import ruptures as rpt
from zsvision.zs_utils import loadmat

In [16]:
class MultiStageModel(nn.Module):
    def __init__(self, num_stages, num_layers, num_f_maps, dim, num_classes):
        super(MultiStageModel, self).__init__()
        self.num_classes = num_classes
        self.stage1 = SingleStageModel(num_layers, num_f_maps, dim, num_classes)
        self.stages = nn.ModuleList([copy.deepcopy(SingleStageModel(num_layers, num_f_maps, num_classes, num_classes)) for s in range(num_stages-1)])

    def forward(self, x, mask):
        out = self.stage1(x, mask)
        outputs = out.unsqueeze(0)
        for s in self.stages:
            if self.num_classes == 1:
                out = s(torch.sigmoid(out) * mask[:, 0:1, :], mask)
            else:
                out = s(F.softmax(out, dim=1) * mask[:, 0:1, :], mask)
            outputs = torch.cat((outputs, out.unsqueeze(0)), dim=0)
        return outputs


class SingleStageModel(nn.Module):
    def __init__(self, num_layers, num_f_maps, dim, num_classes):
        super(SingleStageModel, self).__init__()
        self.num_classes = num_classes
        self.conv_1x1 = nn.Conv1d(dim, num_f_maps, 1)
        self.layers = nn.ModuleList([copy.deepcopy(DilatedResidualLayer(2 ** i, num_f_maps, num_f_maps)) for i in range(num_layers)])
        self.conv_out = nn.Conv1d(num_f_maps, num_classes, 1)

    def forward(self, x, mask):
        out = self.conv_1x1(x)
        for layer in self.layers:
            out = layer(out, mask)
        out = self.conv_out(out) * mask[:, 0:1, :]
        return out


class DilatedResidualLayer(nn.Module):
    def __init__(self, dilation, in_channels, out_channels):
        super(DilatedResidualLayer, self).__init__()
        self.conv_dilated = nn.Conv1d(in_channels, out_channels, 3, padding=dilation, dilation=dilation)
        self.conv_1x1 = nn.Conv1d(out_channels, out_channels, 1)
        self.dropout = nn.Dropout()

    def forward(self, x, mask):
        out = F.relu(self.conv_dilated(x))
        out = self.conv_1x1(out)
        out = self.dropout(out)
        return (x + out) * mask[:, 0:1, :]


In [17]:
def load_mstcn_model(
        mstcn_checkpoint_path: Path,
        device,
        num_blocks: int = 4,
        num_layers: int = 10,
        num_f_maps: int = 64,
        dim: int = 1024,
        num_classes: int = 2,

) -> torch.nn.Module:
    """Load pre-trained MS-TCN checkpoint, put in eval mode."""
    model = MultiStageModel(
        num_blocks, 
        num_layers, 
        num_f_maps, 
        dim, 
        num_classes,
    )

    model = model.to(device)
    checkpoint = torch.load(mstcn_checkpoint_path)
    model.load_state_dict(checkpoint)
    model.eval()
    return model

In [19]:
mstcn_model = torch.load('/kaggle/input/mstscnweights/mstcn_ISL_CMPL.model').to(device)

In [20]:
# mstcn_model = load_mstcn_model(Path("/kaggle/input/mstscnweights/mstcn_ISL_CMPL.model"), device)
def main_mstcn(
    features,
    logits,
    #starting_point: str,
    #i3d_checkpoint_path: Path,
    #mstcn_checkpoint_path: Path,
    #video_path: Path,
    #feature_path: Path,
    #save_path: Path,
    fps: int = 25,
    num_classes: int = 981,
    num_in_frames: int = 16,
    batch_size: int = 16,
    stride: int = 1,
    slowdown_factor: int = 1,
    save_features: bool = True,
    #save_segments: bool = True,
    viz: bool = False,
    generate_vtt: bool = False,
):
#     if (save_segments or generate_vtt) and not save_path.exists():
#         print(f"Creating dir at {save_path}")
#         save_path.mkdir(exist_ok=True, parents=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

#     with BlockTimer("Loading MS-TCN model"):
#         model = load_mstcn_model(
#             mstcn_checkpoint_path=mstcn_checkpoint_path,
#             device=device
#         )
        
    print("Predict segments.")
    sm = nn.Softmax(dim=1)

    # Number of windows/clips
    num_clips = features.shape[0]//100
    # Group the clips into batches
    num_batches = math.ceil(num_clips)

    print(f"num_clips: {num_clips}, num_batches: {num_batches}, num_frames: {features.shape[0]}")

    all_preds = []
    all_probs = []

    for b in range(num_batches+1):
        inp = features[b * 100 : (b + 1) * 100]
        inp = np.swapaxes(inp, 0, 1)
        inp = inp.unsqueeze(0).to(device)
        predictions = mstcn_model(inp, torch.ones(inp.size(), device=device))
        pred_prob = list(sm(predictions[-1]).cpu().detach().numpy())[0][1]
        predicted = torch.tensor(np.where(np.asarray(pred_prob) > 0.5, 1, 0))

        all_preds.extend(torch_to_list(predicted))
        all_probs.extend(pred_prob)
        
    
    return all_preds

In [21]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params


# count_parameters(mstcn_model)

from torchinfo import summary
summary(mstcn_model)

Layer (type:depth-idx)                                  Param #
DataParallel                                            --
├─MultiStageModel: 1-1                                  --
│    └─SingleStageModel: 2-1                            --
│    │    └─Conv1d: 3-1                                 65,600
│    │    └─ModuleList: 3-2                             165,120
│    │    └─Conv1d: 3-3                                 130
│    └─ModuleList: 2-2                                  --
│    │    └─SingleStageModel: 3-4                       165,442
│    │    └─SingleStageModel: 3-5                       165,442
│    │    └─SingleStageModel: 3-6                       165,442
Total params: 727,176
Trainable params: 727,176
Non-trainable params: 0

In [22]:
def get_CP_list(features, pen=80):
    """
    Function for CP Baseline
    Calculate changepoints for whole psiode.

    Args:
        features_dict: dictionary of i3d features to calculate changepoints
        vid_list: list of video names

    Returns:
        dictionary of changepoints (key: video names; value: changepoints)
    """
    CP_list = []
    model ='l2'
    pen = pen
    jump = 2

    if len(features) < 2:
        CP_list = np.zeros(len(features))

    algo = rpt.Pelt(model=model, jump=jump).fit(features)
    res = algo.predict(pen=pen)
    res_np = [1 if ix in res else 0 for ix in range(len(features))]

    CP_list = np.asarray(res_np)

    return CP_list

In [703]:
for videos in tqdm(os.listdir('/home/jupyter/imported/yt_scrape/final_data')):
    if videos[-4:] != '.mp4':
        continue
    
    video_path = Path(os.path.join(dataset_path,videos))
    feature_file = videos[:-4]+'.mat'
    feature_path = Path(os.path.join('/home/jupyter/imported/features-i3d-mstcn', feature_file))
    plpath = Path(os.path.join('/home/jupyter/imported/pseudolabels', feature_file))
    
    if os.path.isfile(plpath):
        continue
    features = torch.tensor(scipy.io.loadmat(feature_path)['preds'])
    pseudolabels = main_mstcn(features, None)
    cp_list = get_CP_list(features)
    
    label_dict = {'PL': pseudolabels, 'CP': cp_list}
    scipy.io.savemat(plpath, mdict=label_dict, do_compression=False, format="4")

In [23]:
class Metric():
    def __init__(self, run_type):
        self.run_type = run_type
        self.overlap = list(np.arange(0.4, 0.76, 0.05))
        self.thresholds_b = list(range(1, 5))

        self.correct = 0
        self.total = 0
        self.width_gt = []
        self.width_pred = []

        self.iou = []
        self.num_det, self.num_gt = 0, 0
        self.dist = []
        self.tp_list = np.zeros(len(self.overlap))
        self.fp_list = np.zeros(len(self.overlap))
        self.fn_list = np.zeros(len(self.overlap))

        self.tp_list_b = np.zeros(len(self.thresholds_b))
        self.fp_list_b = np.zeros(len(self.thresholds_b))
        self.fn_list_b = np.zeros(len(self.thresholds_b))

        self.tp_listm = np.zeros(len(self.overlap))
        self.fp_listm = np.zeros(len(self.overlap))
        self.fn_listm = np.zeros(len(self.overlap))

    def calc_scores_per_batch(self, pred, gt, gt_eval, mask=None):
        if mask is None:
            self.correct += ((pred == gt_eval)).sum().item()
            self.total += gt_eval.shape[0]
        else:
            self.correct += ((pred == gt).float()*mask[:, 0, :].squeeze(1)).sum().item()
            self.total += torch.sum(mask[:, 0, :]).item()

        pred = [item for sublist in torch_to_list(pred) for item in sublist]
        gt_eval = [item for sublist in torch_to_list(gt_eval) for item in sublist]
        gt = [item for sublist in torch_to_list(gt) for item in sublist]

        tp_list1, fp_list1, fn_list1, num_det1, num_gt1, dist1, width_pred1, width_gt1 = get_boundary_metric(pred, gt_eval, self.thresholds_b, bg_class=[0, -100])
        # print(tp_list1, fp_list1, fn_list1, num_det1, num_gt1, dist1, width_pred1, width_gt1)
        self.tp_list_b += tp_list1
        self.fp_list_b += fp_list1
        self.fn_list_b += fn_list1
        self.num_det += num_det1
        self.num_gt += num_gt1
        self.dist.extend(dist1)
        self.width_gt.append(width_gt1)
        self.width_pred.append(width_pred1)

        tp_list1, fp_list1, fn_list1, mean_iou = get_sign_metric(pred, gt, self.overlap, bg_class=[1, -100])
        # print(tp_list1, fp_list1, fn_list1, mean_iou)
        self.iou.append(mean_iou)
        self.tp_list += tp_list1
        self.fp_list += fp_list1
        self.fn_list += fn_list1

    def calc_metrics(self):
        self.mean_iou_ges = np.mean(self.iou)
        self.f1_sign = []
        for s in range(len(self.overlap)):
            self.precision = self.tp_list[s] / float(self.tp_list[s]+self.fp_list[s])
            self.recall = self.tp_list[s] / float(self.tp_list[s]+self.fn_list[s])

            f1 = 2.0 * (self.precision*self.recall) / (self.precision+self.recall)

            f1 = np.nan_to_num(f1)*100
            self.f1_sign.append(round(f1, 2))

        self.mean_f1s = np.mean(self.f1_sign)
        self.f1_sign = [self.f1_sign[2], self.f1_sign[-1]]

        self.f1b = []
        self.recall_list = []
        self.precision_list = []
        for s in range(len(self.thresholds_b)):
            self.precision_b = self.tp_list_b[s] / float(self.tp_list_b[s]+self.fp_list_b[s])
            self.recall_b = self.tp_list_b[s] / float(self.tp_list_b[s]+self.fn_list_b[s])

            f1_b = 2.0 * (self.precision_b*self.recall_b) / (self.precision_b+self.recall_b)

            f1_b = np.nan_to_num(f1_b)*100
            self.f1b.append(f1_b)
            self.recall_list.append(self.recall_b*100)
            self.precision_list.append(self.precision_b*100)

        self.mean_f1b = round(np.mean(self.f1b), 2)
        self.mean_recall_b = round(np.mean(self.recall_list), 2)
        self.mean_precision_b = round(np.mean(self.precision_list), 2)

        self.mean_dist = np.mean(np.abs(self.dist))
        self.mean_width_pred = np.mean(self.width_pred)
        self.mean_width_gt = np.mean(self.width_gt)

        result_dict = {
            'mF1B': self.mean_f1b,
            'F1S': self.f1_sign[-1],
            'mF1S': self.mean_f1s,
            'IoU': 100*self.mean_iou_ges,
            'widthB': self.mean_width_pred,
            'dist': self.mean_dist,
            'detB': self.num_det,
        }
        
        return result_dict


    def save_print_metrics(self, writer, save_dir, epoch, epoch_loss):

        writer.add_scalar(f'{self.run_type}/mF1B', self.mean_f1b, epoch+1)
        writer.add_scalar(f'{self.run_type}/mF1S', self.mean_f1s, epoch+1)

        #add to dict
        result_dict = {epoch: {}}
        result_dict[epoch] = {
            'mF1B': self.mean_f1b,
            'F1S': self.f1_sign[-1],
            'mF1S': self.mean_f1s,
            'IoU': 100*self.mean_iou_ges,
            'widthB': self.mean_width_pred,
            'dist': self.mean_dist,
            'detB': self.num_det,
        }

        if self.run_type == 'train' or self.run_type == 'eval':
            print_str = f"[E{epoch + 1} / {self.run_type}]: epoch loss = {epoch_loss:.4f},   acc = {100*float(self.correct)/self.total:.2f}, mean F1B = {self.mean_f1b:.2f}, mean_F1S = {self.mean_f1s:.2f}"
            save_str = f"[E{epoch + 1} / {self.run_type}]: epoch loss = {epoch_loss:.4f},   acc = {100*float(self.correct)/self.total:.2f}, F1_bound = {self.f1b}, mean_recall_b = {self.mean_recall_b}, mean_precision_b = {self.mean_precision_b}, mean F1B = {self.mean_f1b:.2f}, F1_sign = {self.f1_sign}, mean_F1S = {self.mean_f1s:.2f}, IoU_sign = {100*self.mean_iou_ges:.2f}, #B ({self.num_det}/{self.num_gt}), dist = {self.mean_dist}, boundary width: {self.mean_width_pred:.2f} / {self.mean_width_gt:.2f}  \n"

            if self.run_type == 'train':
                print(f'\n{print_str}')
            else:
                print(f'{print_str}\n')

            with open(f'{save_dir}/train_progress.txt', 'a+') as f:
                if self.run_type == 'train':
                    f.write('\n\n ---------------------------------------------------\n')
                f.write(save_str)

        elif self.run_type == 'test':
            save_dir = f'{str(Path(save_dir).parent)}/{str(Path(save_dir).stem)}.txt'

            print(f"Acc: {(100*float(self.correct)/self.total):.2f}")
            print(f"F1B: {(self.f1b)}")
            print(f"mean Recall B: {(self.mean_recall_b)}")
            print(f"mean Precision B: {(self.mean_precision_b)}")
            print(f'mean F1B: {(self.mean_f1b):.2f}')
            print(f'IoU Sign: {(100*self.mean_iou_ges):.2f}')
            print(f'F1s: {(self.f1_sign)}')
            print(f'mean F1S: {(self.mean_f1s):.2f}')
            print(f'#B:  ({self.num_det}/{self.num_gt})')
            print(f'mean dist [frames]: {self.mean_dist:.2f}')
            print(f'mean B width pred/gt [frames]: {self.mean_width_pred:.2f} / {self.mean_width_gt:.2f} \n')

            with open(save_dir, 'a+') as f:
                f.write(f'Acc: {(100*float(self.correct)/self.total):.2f} \n')
                f.write(f'F1b: {(self.f1b)} \n')
                f.write(f'mean Recall B: {(self.mean_recall_b):.2f} \n')
                f.write(f'mean Precision B: {(self.mean_precision_b):.2f} \n')
                f.write(f'mean F1b: {(self.mean_f1b):.2f} \n')
                f.write(f'IoU Sign: {(100*self.mean_iou_ges):.2f} \n')
                f.write(f'F1s: {(self.f1_sign)} \n')
                f.write(f'mean F1S: {(self.mean_f1s):.2f} \n')
                f.write(f'#B:  ({self.num_det}/{self.num_gt})  \n')
                f.write(f'mean dist [frames]: {self.mean_dist:.2f} \n')
                f.write(f'mean B width pred/gt [frames]: {self.mean_width_pred:.2f} / {self.mean_width_gt:.2f} \n')

        return result_dict


def get_boundary_metric(pred, gt, thresholds, bg_class=[0]):

    p_label, p_start, p_end = get_labels_start_end_time(pred, bg_class)
    y_label, y_start, y_end = get_labels_start_end_time(gt, bg_class)
    # print(p_label, p_start, p_end)
    # print(y_label, y_start, y_end)

    mean_boundary_width_pred = list(pred).count(1) / len(p_label) if len(p_label) else 0
    mean_boundary_width_gt = list(gt).count(1) / len(y_label) if len(y_label) else 0

    num_pred = len(p_label)
    num_gt = len(y_label)

    pos_p = [(p_end[i]+p_start[i])/2 for i in range(len(p_label))]
    pos_y = [(y_end[i]+y_start[i])/2 for i in range(len(y_label))]

    # calculate distance matrix
    if len(p_label) > 0 and len(y_label) > 0:
        dist_all = []
        for p in pos_p:
            dist_all.append([abs(y-p) for y in pos_y])
        dist_arr = np.asarray(dist_all)

        # calculate mean distance
        # print(dist_arr)
        mean_dist = [np.mean(np.min(dist_arr, 1))]
    else:
        mean_dist = [0]

    # find smallest distances
    dist_choosen = []
    if len(p_label) > 0 and len(y_label) > 0:
        for ix in range(min(dist_arr.shape[0], dist_arr.shape[1])):

            argmin_row = np.argmin(dist_arr, axis=1)
            min_row = np.min(dist_arr, axis=1)
            min_dist = np.min(min_row)
            argmin_dist = np.argmin(min_row)

            dist_choosen.append(min_dist)

            # delete row and column -> pred-gt pair can't be reused
            dist_arr = np.delete(dist_arr, argmin_dist, 0)
            dist_arr = np.delete(dist_arr, argmin_row[argmin_dist], 1)

    tp_list = []
    fp_list = []
    fn_list = []

    for th in thresholds:
        tp = 0
        fp = 0
        for dist in dist_choosen:
            if dist <= th:
                tp += 1
            else:
                fp += 1

        # more predictions than gt -> count as false positiv
        fp += max(0, len(p_label)-len(dist_choosen))
        # difference between number of true boundaries and correct predicted ones -> false negative
        fn = len(y_label) - tp

        tp_list.append(tp)
        fp_list.append(fp)
        fn_list.append(fn)

    return np.asarray(tp_list), np.asarray(fp_list), np.asarray(fn_list), num_pred, num_gt, mean_dist, mean_boundary_width_pred, mean_boundary_width_gt


def get_sign_metric(pred, gt, overlap, bg_class=[1]):
    p_label, p_start, p_end = get_labels_start_end_time(pred, bg_class)
    y_label, y_start, y_end = get_labels_start_end_time(gt, bg_class)
    iou_all = []

    for j in range(len(p_label)):
        intersection = np.minimum(p_end[j], y_end) - np.maximum(p_start[j], y_start)
        union = np.maximum(p_end[j], y_end) - np.minimum(p_start[j], y_start)
        iou_all.append((1.0*intersection / union)*([p_label[j] == y_label[x] for x in range(len(y_label))]))

    iou_arr = np.asarray(iou_all)
    iou_choosen = []
    if len(p_label) > 0:
        for ix in range(min(iou_arr.shape[0], iou_arr.shape[1])):
            argmax_row = np.argmax(iou_arr, axis=1)
            max_row = np.max(iou_arr, axis=1)
            max_iou = np.max(max_row)
            argmax_iou = np.argmax(max_row)
            iou_choosen.append(max_iou)
            iou_arr = np.delete(iou_arr, argmax_iou, 0)
            iou_arr = np.delete(iou_arr, argmax_row[argmax_iou], 1)

        diff = max(iou_arr.shape[0], iou_arr.shape[1]) - len(iou_choosen)
    else:
        diff = len(y_label)

    tp_list = []
    fp_list = []
    fn_list = []

    for ol in overlap:
        tp = 0
        fp = 0
        for match in iou_choosen:
            if match > ol:
                tp += 1
            else:
                fp += 1
        fp += max(0, len(p_label)-len(iou_choosen))
        fn = len(y_label) - tp
        tp_list.append(tp)
        fp_list.append(fp)
        fn_list.append(fn)

    iou_choosen.extend([0]*diff)
    mean_iou = np.mean(iou_choosen)

    return np.asarray(tp_list), np.asarray(fp_list), np.asarray(fn_list), mean_iou

In [24]:
# PL = batch_target.cpu().numpy()
# CP = predicted.numpy()
def merge_PL_CP(PL, CP):

    merges = np.array(CP)|np.array(PL)

    return merges

def CMPL(PL, CP):

    SL_insertion = np.asarray(PL.copy())
    CMSL_th_refine = 4000
    CMSL_th_insert = 4000
    _, PL_starts, PL_ends = get_labels_start_end_time(PL, [0])
    _, CP_starts, _ = get_labels_start_end_time(CP, [0])

    if CMSL_th_insert < 9000:
        for cp_pos in CP_starts:
            insert = True
            for PL_start, PL_end in zip(PL_starts, PL_ends):
                if abs(PL_start-cp_pos) < CMSL_th_insert:
                    insert = False
                if abs(cp_pos-PL_end) < CMSL_th_insert:
                    insert = False
            if insert == True:
                SL_insertion[cp_pos-1:cp_pos+2] = 1


    # refinement
    SL_refinement = SL_insertion.copy()

    if CMSL_th_refine < 9000:
        _, PL_starts, PL_ends = get_labels_start_end_time(SL_refinement, [0])
        _, CP_starts, CP_ends = get_labels_start_end_time(CP, [0])
        for cp_pos in CP_starts:
            refine = False
            PL_ix = -1
            dist = 1000
            for ix, (PL_start, PL_end) in enumerate(zip(PL_starts, PL_ends)):
                if abs(PL_start-cp_pos) < CMSL_th_refine and abs(PL_start-cp_pos)<dist:
                    refine = True
                    dist = abs(PL_start-cp_pos)
                    PL_ix = ix
                if abs(cp_pos-PL_end) < CMSL_th_refine and abs(cp_pos-PL_end)<dist:
                    refine = True
                    dist = abs(cp_pos-PL_end)
                    PL_ix = ix

            if refine == True:
                mean_ps = (PL_ends[PL_ix] + PL_starts[PL_ix]) // 2
                new_pos = (mean_ps + cp_pos) // 2
                SL_refinement[PL_starts[PL_ix]:PL_ends[PL_ix]] = 0
                SL_refinement[new_pos-1:new_pos+2] = 1

    return SL_refinement

In [955]:
# import pandas as pd
# cpdict = pd.read_pickle(r'/home/jupyter/imported/test/cp_dict.pkl')
# pldict = pd.read_pickle(r'/home/jupyter/imported/test/pl_dict.pkl')

# for key in cpdict.keys():
#     # print(key)
#     save_path = os.path.join('/home/jupyter/imported/test/pseudolabels', key + '.mat')
#     pseudolabels = pldict[key]
#     cp_list = cpdict[key]

#     label_dict = {'PL': pseudolabels, 'CP': cp_list}
#     scipy.io.savemat(save_path, mdict=label_dict, do_compression=False, format="4")
# # len(obj.keys())

In [26]:
import torch.optim as optim
print(device)

mstcn_checkpoint_path = Path("/kaggle/input/mstscnweights/mstcn_ISL_CMPL.model")
num_blocks = 4
num_layers = 10
num_f_maps = 64
dim = 1024
num_classes = 2

# mstcn_trainable = MultiStageModel(num_blocks, num_layers, num_f_maps, dim, num_classes)
# mstcn_trainable.train()
# mstcn_trainable = mstcn_trainable.to(device)
# checkpoint = torch.load(mstcn_checkpoint_path)
# mstcn_trainable.load_state_dict(checkpoint)

mstcn_trainable = torch.load('/kaggle/input/mstscnweights/mstcn_ISL_CMPL.model')

# mstcn_trainable = load_mstcn_model(Path("/home/jupyter/imported/models/ms-tcn/mstcn_bslcp_i3d_bslcp.model"), device)
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    mstcn_trainable = nn.DataParallel(mstcn_trainable)

ce = nn.CrossEntropyLoss()
mse = nn.MSELoss(reduction='none')
mse_red = nn.MSELoss(reduction='mean')

# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
optimizer = optim.Adam(mstcn_trainable.parameters(), lr=0.0005, betas=(0.9, 0.999), eps=1e-8)

cuda


In [28]:
from sklearn.metrics import f1_score
def flatten(l):
    return [item for sublist in l for item in sublist]

def train(model, dataloader):
    model.train()
    running_loss = 0
    total = 0
    correct = 0
    l_preds = []
    l_labels = []
    metric = Metric('train')

    with tqdm(dataloader) as t:
        t.set_description('Training')
        for i, (features, labels) in enumerate(t):  
            features, labels = features.to(device), labels.type(torch.LongTensor).to(device)
            optimizer.zero_grad()
            
            inp = features.permute(0,2,1)
            out = mstcn_trainable(inp, torch.ones(inp.size(), device=device))
            # print(out.shape)
            out_ll = out[-1]
            pred_prob = F.softmax(out_ll, dim=1)[0][1]

            # loss = criterion(out_ll[0][1], labels.squeeze().float())

            loss = 0
            batch_target = labels.squeeze()
            # mask = torch.ones_like(inp, dtype=torch.float)
            for ix, p in enumerate(out):
                if num_classes == 1:
                    loss += mse_red(p.transpose(2, 1).contiguous().view(-1, num_classes).squeeze(), batch_target.view(-1))
                else:
                    # print(p.shape)
                    # print(p.transpose(2, 1).contiguous().view(-1, num_classes).shape)
                    # print(batch_target.shape)
                    loss += ce(p.transpose(2, 1).contiguous().view(-1, num_classes), batch_target.view(-1))
                    loss += 0.15*torch.mean(torch.clamp(mse(F.log_softmax(p[:, :, 1:], dim=1), F.log_softmax(p.detach()[:, :, :-1], dim=1)), min=0, max=16))

            running_loss += loss.item()
            t.set_postfix({'loss':running_loss/(i+1)})
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            predicted = torch.tensor(np.where(np.asarray(pred_prob.cpu().detach().numpy()) > 0.5, 1, 0))

            total += batch_target.cpu().size(0)
            correct += (predicted == batch_target.cpu()).sum().item()
            
            metric.calc_scores_per_batch(predicted.to(device).unsqueeze(0), batch_target.unsqueeze(0),  batch_target.unsqueeze(0), None)
            
            l_preds.append(predicted.cpu().numpy())
            l_labels.append(batch_target.cpu().numpy())
    result = metric.calc_metrics()
    return 100*correct/total, running_loss/len(dataloader), result

def validate(model, dataloader):
    model.eval()
    running_loss = 0
    total = 0
    correct = 0
    l_preds = []
    l_labels = []
    metric = Metric('test')

    with torch.no_grad():
        with tqdm(dataloader) as t:
            t.set_description('Validating')
            for i, (features, labels) in enumerate(t):  
                features, labels = features.to(device), labels.type(torch.LongTensor).to(device)
                inp = features.permute(0,2,1)
                out = mstcn_trainable(inp, torch.ones(inp.size(), device=device))
                # print(out.shape)
                out_ll = out[-1]
                pred_prob = F.softmax(out_ll, dim=1)[0][1]

                loss = 0
                batch_target = labels.squeeze()
                for ix, p in enumerate(out):
                    if num_classes == 1:
                        loss += mse_red(p.transpose(2, 1).contiguous().view(-1, num_classes).squeeze(), batch_target.view(-1))
                    else:
                        loss += ce(p.transpose(2, 1).contiguous().view(-1, num_classes), batch_target.view(-1))
                        loss += 0.15*torch.mean(torch.clamp(mse(F.log_softmax(p[:, :, 1:], dim=1), F.log_softmax(p.detach()[:, :, :-1], dim=1)), min=0, max=16))

                running_loss += loss.item()
                t.set_postfix({'loss':running_loss/(i+1)})
                running_loss += loss.item()
                predicted = torch.tensor(np.where(np.asarray(pred_prob.cpu().detach().numpy()) > 0.5, 1, 0))

                total += batch_target.cpu().size(0)
                correct += (predicted == batch_target.cpu()).sum().item()
                # print(predicted)
                metric.calc_scores_per_batch(predicted.to(device).unsqueeze(0), batch_target.unsqueeze(0),  batch_target.unsqueeze(0), None)

                l_preds.append(predicted.cpu().numpy())
                l_labels.append(batch_target.cpu().numpy())
    result = metric.calc_metrics()

    return 100*correct/total, running_loss/len(dataloader), result


In [958]:
# CMPL(PL.numpy(), CP)

In [29]:
from torch.utils.data import Dataset, DataLoader

class ISLDataset(Dataset):

    def __init__(self, feature_dir, labels_dir, strategy='merge', mode='train'):
        self.feature_dir = feature_dir
        self.labels_dir = labels_dir
        video_names = [i[:-4] for i in os.listdir(self.feature_dir) if i.endswith('.mat')]
        self.feature_files = [os.path.join(self.feature_dir, video_names[i]) for i in range(len(video_names))]
        self.label_files = [os.path.join(self.labels_dir, video_names[i]) for i in range(len(video_names))]
        self.strategy = strategy
        self.mode = mode

    def __len__(self):
        return len(self.feature_files)

    def __getitem__(self, idx):
        
        try:
            features = torch.tensor(scipy.io.loadmat(self.feature_files[idx])['preds'])
        except:
            features = torch.tensor(scipy.io.loadmat(self.feature_files[idx])['features'])
        
        # if self.mode == 'test':
        #     # print("here")
        #     labels = torch.tensor(scipy.io.loadmat(self.label_files[idx])['gt']).long()
        #     # print(features.shape, labels.shape)
        #     if features.shape[0] > labels.shape[1]:
        #         features = features[:labels.shape[1], :]
        #     else:
        #         labels = labels[:,:features.shape[0]]
        #     return features, torch.tensor(labels)
        
        PL = torch.tensor(scipy.io.loadmat(self.label_files[idx])['PL']).long()
        CP = torch.tensor(scipy.io.loadmat(self.label_files[idx])['CP']).long()
        # print(PL.shape, CP.shape)
        if self.strategy == 'merge':
            labels = merge_PL_CP(PL, CP)
        elif self.strategy == 'CMPL':
            labels = CMPL(PL.squeeze(0).numpy(), CP.squeeze(0).numpy())
            labels = torch.tensor(labels).unsqueeze(0)
        elif self.strategy == 'PL':
            labels = PL
        elif self.strategy == 'CP':
            labels = CP
        else:
            print('No such strategy Exists')
    
        return features, torch.tensor(labels)

In [977]:
# plt.figure(figsize=(12,4))
# plt.plot(CMPL(PL, CP), alpha=0.3)
# # plt.plot(CP, alpha=0.1)
# # plt.plot(PL, alpha=0.1)
# plt.plot(merge_PL_CP(PL,CP), alpha=0.3)
# plt.xlim([0,100])
# plt.show()


In [969]:
def get_dataloaders(strategy='merge'):
    feature_dir = '/kaggle/input/testingdata/i3d_features/i3d_features'
    labels_dir = '/home/jupyter/imported/pseudolabels'
    dataset = ISLDataset(feature_dir, labels_dir, strategy)
    train_size = int(1*len(dataset))
    val_size = len(dataset) - train_size
    train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])
    train_dataloader = DataLoader(train_set, batch_size=1, shuffle=True)
    val_dataloader = DataLoader(val_set, batch_size=1, shuffle=False)

    feature_dir = '/home/jupyter/imported/test/output'
    labels_dir = '/home/jupyter/imported/test/ground_truth'
    test_dataset = ISLDataset(feature_dir, labels_dir, strategy, 'test')
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    return train_dataloader, test_dataloader

In [970]:
train_dataloader, test_dataloader = get_dataloaders('merge')
for features, labels in test_dataloader:
    print(features.shape, labels.shape)
    break

torch.Size([1, 94, 1024]) torch.Size([1, 1, 94])


In [971]:
import pickle

In [972]:
def run_model(strategy='merge'):
    train_dataloader, test_dataloader = get_dataloaders(strategy)
    num_epochs = 10
    list_train_results = []
    list_test_results = []
    for epoch in range(num_epochs):
        train_acc, train_loss, train_result = train(mstcn_trainable, train_dataloader)
        test_acc, test_loss, test_result = validate(mstcn_trainable, test_dataloader)
        print(f"Epoch {epoch + 1}: Train Loss = {train_loss:.4f}, Test Loss = {test_loss:.4f},  Train Acc= {train_acc:.2f}%, Test Acc = {test_acc:.2f}%")
        print(train_result, test_result)
        list_test_results.append(test_result)
        list_train_results.append(train_result)
    
#     result_save_dir = '/home/jupyter/imported/results'
#     test_save = os.path.join(result_save_dir, f'{num_epochs}epochs-{strategy}-bslcp-isl-test-lr0.005.pkl')
#     with open(test_save, 'wb') as f:
#         pickle.dump(list_test_results, f)
        
#     train_save = os.path.join(result_save_dir, f'{num_epochs}epochs-{strategy}-bslcp-isl-train-lr0.005.pkl')
#     with open(train_save, 'wb') as f:
#         pickle.dump(list_train_results, f)


In [973]:
def run_test(strategy='merge'):
    train_dataloader, test_dataloader = get_dataloaders(strategy)
    list_test_results = []
    for epoch in range(1):
        test_acc, test_loss, test_result = validate(mstcn_trainable, test_dataloader)
        print(f"Epoch {epoch + 1}: Val Loss = {test_loss:.4f},  Val Acc = {test_acc:.2f}%")
        list_test_results.append(test_result)
        # print(test_result)
        print(f'Strategy: {strategy}, mF1B: {test_result["mF1B"]}, mF1S: {test_result["mF1S"]}')
        
    
    result_save_dir = '/home/jupyter/imported/results'
    test_save = os.path.join(result_save_dir, f'no-train-{strategy}-bslcp-isl-test.pkl')
    with open(test_save, 'wb') as f:
        pickle.dump(list_test_results, f)

In [974]:
run_test('CMPL')

Validating: 100%|██████████| 137/137 [00:03<00:00, 45.29it/s, loss=6.1] 


Epoch 1: Val Loss = 6.1240,  Val Acc = 59.36%

Strategy: CMPL, mF1B: 19.34, mF1S: 7.16625


In [975]:
run_model(strategy='CMPL')

Training: 100%|██████████| 698/698 [00:46<00:00, 15.01it/s, loss=3.64]

Validating: 100%|██████████| 137/137 [00:03<00:00, 45.39it/s, loss=5.45]


Epoch 1: Train Loss = 3.6387, Test Loss = 5.4647,  Train Acc= 80.89%, Test Acc = 63.02%

{'mF1B': 63.23, 'F1S': 31.22, 'mF1S': 50.4975, 'IoU': 57.7838078656754, 'widthB': 2.3587327051444333, 'dist': 1.7048503757611895, 'detB': 16926} {'mF1B': 20.08, 'F1S': 1.51, 'mF1S': 9.7425, 'IoU': 20.117692447835225, 'widthB': 3.060625303326033, 'dist': 12.12233464021785, 'detB': 1088}


Training: 100%|██████████| 698/698 [00:47<00:00, 14.80it/s, loss=3.47]

Validating: 100%|██████████| 137/137 [00:03<00:00, 45.36it/s, loss=5.61]


Epoch 2: Train Loss = 3.4755, Test Loss = 5.6309,  Train Acc= 82.11%, Test Acc = 61.81%

{'mF1B': 67.73, 'F1S': 36.42, 'mF1S': 55.69499999999999, 'IoU': 62.63573705989938, 'widthB': 2.5434079446452906, 'dist': 1.5970849354895231, 'detB': 17948} {'mF1B': 19.86, 'F1S': 1.8, 'mF1S': 9.67875, 'IoU': 19.63087150891032, 'widthB': 3.3649610324792802, 'dist': 12.381456398427202, 'detB': 1094}


Training: 100%|██████████| 698/698 [00:47<00:00, 14.84it/s, loss=3.41]

Validating: 100%|██████████| 137/137 [00:02<00:00, 45.97it/s, loss=5.33]


Epoch 3: Train Loss = 3.4147, Test Loss = 5.3420,  Train Acc= 82.66%, Test Acc = 67.24%

{'mF1B': 69.14, 'F1S': 38.59, 'mF1S': 57.24875, 'IoU': 63.584788199913724, 'widthB': 2.616090715010656, 'dist': 1.5415450251073823, 'detB': 18213} {'mF1B': 19.85, 'F1S': 6.93, 'mF1S': 16.87375, 'IoU': 31.953695053941004, 'widthB': 2.6908103453723893, 'dist': 12.149886376802435, 'detB': 815}


Training: 100%|██████████| 698/698 [00:47<00:00, 14.78it/s, loss=3.36]

Validating: 100%|██████████| 137/137 [00:03<00:00, 44.82it/s, loss=5.86]


Epoch 4: Train Loss = 3.3673, Test Loss = 5.8837,  Train Acc= 82.99%, Test Acc = 58.90%

{'mF1B': 70.09, 'F1S': 39.7, 'mF1S': 58.615, 'IoU': 64.16099694369714, 'widthB': 2.6716121406606366, 'dist': 1.5062140545175313, 'detB': 18365} {'mF1B': 20.01, 'F1S': 1.18, 'mF1S': 6.1875, 'IoU': 15.74614960066809, 'widthB': 3.464370098676668, 'dist': 12.37027337018213, 'detB': 1176}


Training:  52%|█████▏    | 366/698 [00:24<00:22, 14.74it/s, loss=3.31]


KeyboardInterrupt: 

In [807]:
run_model(strategy='PL')

Training: 100%|██████████| 698/698 [00:45<00:00, 15.23it/s, loss=2.75]

Validating: 100%|██████████| 137/137 [00:03<00:00, 43.59it/s, loss=7.54]


Epoch 1: Train Loss = 2.7555, Val Loss = 10.1234,  Train Acc= 87.87%, Val Acc = 53.07%

{'mF1B': 80.86, 'F1S': 54.69, 'mF1S': 70.2125, 'IoU': 69.6306939983843, 'widthB': 4.308285687505924, 'dist': 1.3113739556341857, 'detB': 19793} {'mF1B': 50.87, 'F1S': 9.08, 'mF1S': 23.21125, 'IoU': 35.81304617841543, 'widthB': 4.704400626566793, 'dist': 4.136861057696181, 'detB': 1243}


Training: 100%|██████████| 698/698 [00:46<00:00, 15.14it/s, loss=2.66]

Validating: 100%|██████████| 137/137 [00:03<00:00, 43.51it/s, loss=8.08]


Epoch 2: Train Loss = 2.6611, Val Loss = 10.1234,  Train Acc= 88.77%, Val Acc = 53.07%

{'mF1B': 82.25, 'F1S': 57.23, 'mF1S': 71.9175, 'IoU': 71.63440134337877, 'widthB': 4.337828856746114, 'dist': 1.1814736658334453, 'detB': 19869} {'mF1B': 49.86, 'F1S': 8.22, 'mF1S': 23.45625, 'IoU': 35.85851102688971, 'widthB': 5.07313297003078, 'dist': 4.1661590234582935, 'detB': 1181}


Training: 100%|██████████| 698/698 [00:46<00:00, 15.16it/s, loss=2.61]

Validating: 100%|██████████| 137/137 [00:03<00:00, 45.16it/s, loss=8.38]


...

Epoch 8: Train Loss = 2.4666, Val Loss = 10.1234,  Train Acc= 90.73%, Val Acc = 53.07%

{'mF1B': 85.33, 'F1S': 62.9, 'mF1S': 76.255, 'IoU': 75.80863886882801, 'widthB': 4.371553391192509, 'dist': 0.9832151063479464, 'detB': 20117} {'mF1B': 48.29, 'F1S': 10.62, 'mF1S': 29.54875, 'IoU': 41.59405029914709, 'widthB': 4.786279094694724, 'dist': 4.072021141160257, 'detB': 1043}


Training: 100%|██████████| 698/698 [00:46<00:00, 15.02it/s, loss=2.44]

Validating: 100%|██████████| 137/137 [00:03<00:00, 44.65it/s, loss=6.1] 


Epoch 9: Train Loss = 2.4451, Val Loss = 10.1234,  Train Acc= 90.96%, Val Acc = 53.07%

{'mF1B': 85.59, 'F1S': 63.76, 'mF1S': 76.81, 'IoU': 76.46797209418078, 'widthB': 4.374755695957915, 'dist': 0.9532550467466455, 'detB': 20121} {'mF1B': 48.42, 'F1S': 12.28, 'mF1S': 30.42375, 'IoU': 43.20776924693815, 'widthB': 4.5434780547919225, 'dist': 4.002806479813779, 'detB': 990}


Training: 100%|██████████| 698/698 [00:46<00:00, 15.14it/s, loss=2.43]

Validating: 100%|██████████| 137/137 [00:03<00:00, 42.27it/s, loss=7.05]

Epoch 10: Train Loss = 2.4283, Val Loss = 10.1234,  Train Acc= 91.19%, Val Acc = 53.07%

{'mF1B': 85.71, 'F1S': 63.87, 'mF1S': 77.26875, 'IoU': 76.52786321918552, 'widthB': 4.373280087577134, 'dist': 0.9503815733153755, 'detB': 20165} {'mF1B': 48.02, 'F1S': 10.61, 'mF1S': 28.311249999999998, 'IoU': 41.36079738647267, 'widthB': 5.098216544201946, 'dist': 4.08942995366353, 'detB': 1061}


In [810]:
run_model(strategy='CP')

Training: 100%|██████████| 698/698 [00:44<00:00, 15.59it/s, loss=1.68]

Validating: 100%|██████████| 137/137 [00:03<00:00, 44.63it/s, loss=8.91]


Epoch 1: Train Loss = 1.6856, Val Loss = 10.1234,  Train Acc= 94.47%, Val Acc = 53.07%

{'mF1B': 0.55, 'F1S': 0.17, 'mF1S': 0.39875000000000005, 'IoU': 0.6735943223173538, 'widthB': 0.024290714891188295, 'dist': 0.0423458740918564, 'detB': 118} {'mF1B': 0.0, 'F1S': 0.0, 'mF1S': 0.79625, 'IoU': 7.04150269862757, 'widthB': 0.0, 'dist': 0.0, 'detB': 0}


Training: 100%|██████████| 698/698 [00:44<00:00, 15.59it/s, loss=1.61]

Validating: 100%|██████████| 137/137 [00:03<00:00, 45.08it/s, loss=7.59]


Epoch 2: Train Loss = 1.6085, Val Loss = 10.1234,  Train Acc= 94.57%, Val Acc = 53.07%

{'mF1B': 0.0, 'F1S': 0.0, 'mF1S': 0.0025, 'IoU': 0.42797154868335385, 'widthB': 0.0, 'dist': 0.0, 'detB': 0} {'mF1B': 0.0, 'F1S': 0.0, 'mF1S': 0.79625, 'IoU': 7.04150269862757, 'widthB': 0.0, 'dist': 0.0, 'detB': 0}


Training: 100%|██████████| 698/698 [00:44<00:00, 15.71it/s, loss=1.58]

Validating: 100%|██████████| 137/137 [00:03<00:00, 43.72it/s, loss=34.2]


...

Epoch 7: Train Loss = 1.5177, Val Loss = 10.1234,  Train Acc= 94.57%, Val Acc = 53.07%

{'mF1B': 0.41, 'F1S': 0.04, 'mF1S': 0.07250000000000001, 'IoU': 0.5154787782204888, 'widthB': 0.0419054441260745, 'dist': 0.033369149952244506, 'detB': 39} {'mF1B': 0.0, 'F1S': 0.0, 'mF1S': 0.79625, 'IoU': 7.04150269862757, 'widthB': 0.0, 'dist': 0.0, 'detB': 0}


Training: 100%|██████████| 698/698 [00:44<00:00, 15.73it/s, loss=1.52]

Validating: 100%|██████████| 137/137 [00:03<00:00, 43.74it/s, loss=8.28]


Epoch 8: Train Loss = 1.5259, Val Loss = 10.1234,  Train Acc= 94.57%, Val Acc = 53.07%

{'mF1B': 0.14, 'F1S': 0.01, 'mF1S': 0.02, 'IoU': 0.4571778504875131, 'widthB': 0.015759312320916905, 'dist': 0.013610315186246419, 'detB': 13} {'mF1B': 0.0, 'F1S': 0.0, 'mF1S': 0.79625, 'IoU': 7.04150269862757, 'widthB': 0.0, 'dist': 0.0, 'detB': 0}


Training:  42%|████▏     | 295/698 [00:18<00:25, 15.57it/s, loss=1.54]


KeyboardInterrupt: 

In [ ]:
mstcn_trainable.state_dict()

# Trying to get scores on the testing data

In [127]:
import pickle
from torch.utils.data import TensorDataset, DataLoader
import statistics

In [74]:
def validate(model, dataloader):
    model.eval()
    running_loss = 0
    total = 0
    correct = 0
    l_preds = []
    l_labels = []
    metric = Metric('test')

    with torch.no_grad():
        with tqdm(dataloader) as t:
            t.set_description('Validating')
            for i, (features, labels) in enumerate(t):  
                features, labels = features.to(device), labels.type(torch.LongTensor).to(device)
                inp = features.permute(0,2,1)
                out = mstcn_trainable(inp, torch.ones(inp.size(), device=device))
                # print(out.shape)
                out_ll = out[-1]
                pred_prob = F.softmax(out_ll, dim=1)[0][1]

                loss = 0
                batch_target = labels.squeeze()
                for ix, p in enumerate(out):
                    if num_classes == 1:
                        loss += mse_red(p.transpose(2, 1).contiguous().view(-1, num_classes).squeeze(), batch_target.view(-1))
                    else:
                        loss += ce(p.transpose(2, 1).contiguous().view(-1, num_classes), batch_target.view(-1))
                        loss += 0.15*torch.mean(torch.clamp(mse(F.log_softmax(p[:, :, 1:], dim=1), F.log_softmax(p.detach()[:, :, :-1], dim=1)), min=0, max=16))

                running_loss += loss.item()
                t.set_postfix({'loss':running_loss/(i+1)})
                running_loss += loss.item()
                predicted = torch.tensor(np.where(np.asarray(pred_prob.cpu().detach().numpy()) > 0.5, 1, 0))

                total += batch_target.cpu().size(0)
                correct += (predicted == batch_target.cpu()).sum().item()
                # print(predicted)
                metric.calc_scores_per_batch(predicted.to(device).unsqueeze(0), batch_target.unsqueeze(0),  batch_target.unsqueeze(0), None)

                l_preds.append(predicted.cpu().numpy())
                l_labels.append(batch_target.cpu().numpy())
    result = metric.calc_metrics()

    return 100*correct/total, running_loss/len(dataloader), result


In [182]:
with open('/kaggle/input/framedict/frameDict.pkl', 'rb') as f:
    ground_truth_test = pickle.load(f)
    
print(type(ground_truth_test))
print(len(ground_truth_test))

<class 'dict'>
176


In [183]:
class Test_dataset(Dataset):
    
    def __init__(self, feature_path, label_path):
        super().__init__()
        self.feature_path = feature_path # path to .mat file
        self.label_path = label_path # key name to get the gt labels from the dict 
        
    def __len__(self):
        return 1
    
    def __getitem__(self, index):
        
        i3d_features_tensor = torch.tensor(scipy.io.loadmat(self.feature_path)['features'], dtype=torch.float32)
        gt_tensor = torch.tensor(list(ground_truth_test[self.label_path].values()))
        
        len_gt = len(gt_tensor)
        len_i3d = len(i3d_features_tensor)
        
        if len_gt < len_i3d:
            i3d_features_tensor = i3d_features_tensor[:len_gt, :]
        elif len_gt >= len_i3d:
            gt_tensor = gt_tensor[:len_i3d]
        
        assert len(gt_tensor)==len(i3d_features_tensor)
        
        return i3d_features_tensor, gt_tensor

In [184]:
my_path = '/kaggle/input/testingdata/i3d_features/i3d_features'
mat_files_path = os.listdir(my_path)
mat_files_path = [os.path.join(my_path, x) for x in mat_files_path if x[-4:] == '.mat']
vid_files_path = []

for x in mat_files_path:
    x = str(x).split('/')[-1][:-4]
    vid_files_path.append(x)
    

print(len(mat_files_path), len(vid_files_path))
print(mat_files_path[0])
print(vid_files_path[0])

176 176
/kaggle/input/testingdata/i3d_features/i3d_features/could you please talk slower (5).MP4.mat
could you please talk slower (5).MP4


In [242]:
# results of the trained model

mF1B_list = []
mF1S_list = []

for i in tqdm(range(len(mat_files_path))):
    my_dat = Test_dataset(mat_files_path[i], vid_files_path[i])
    my_loader = DataLoader(my_dat, batch_size=1)
    loss, acc, result = validate(new_model, my_loader)
    mF1B_list.append(result['mF1B'])
    mF1S_list.append(result['mF1S'])
    
print(len(mF1B_list), len(mF1S_list))

  0%|          | 0/176 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

176 176


In [243]:
mF1B_mean = statistics.mean(mF1B_list)
mF1S_mean = statistics.mean(mF1S_list)

print(f"mF1B = {mF1B_mean}, mF1S = {mF1S_mean}")

mF1B = 7.814034090909091, mF1S = 33.38460227272727


In [244]:
max(mF1B_list), max(mF1S_list)

(66.67, 100.0)

In [239]:
my_model = torch.load('/kaggle/input/mstscnweights/mstcn_ISL_CMPL.model')
# results of the model without training

mF1B_list = []
mF1S_list = []

for i in tqdm(range(len(mat_files_path))):
    my_dat = Test_dataset(mat_files_path[i], vid_files_path[i])
    my_loader = DataLoader(my_dat, batch_size=1)
    loss, acc, result = validate(my_model, my_loader)
    mF1B_list.append(result['mF1B'])
    mF1S_list.append(result['mF1S'])
    
print(len(mF1B_list), len(mF1S_list))

  0%|          | 0/176 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

176 176


In [240]:
mF1B_mean = statistics.mean(mF1B_list)
mF1S_mean = statistics.mean(mF1S_list)

print(f"mF1B = {mF1B_mean}, mF1S = {mF1S_mean}")

mF1B = 7.814034090909091, mF1S = 33.38460227272727


In [241]:
max(mF1B_list), max(mF1S_list)

(66.67, 100.0)

In [221]:
mF1B_list

[0.0,
 0.0,
 0.0,
 20.0,
 0.0,
 0.0,
 0.0,
 0.0,
 12.5,
 0.0,
 40.0,
 0.0,
 0.0,
 0.0,
 25.0,
 0.0,
 50.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 25.0,
 0.0,
 66.67,
 25.0,
 0.0,
 50.0,
 37.5,
 0.0,
 0.0,
 0.0,
 0.0,
 40.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 25.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 33.33,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 25.0,
 12.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 25.0,
 0.0,
 0.0,
 0.0,
 0.0,
 25.0,
 0.0,
 0.0,
 28.57,
 0.0,
 25.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 12.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 50.0,
 0.0,
 0.0,
 50.0,
 0.0,
 0.0,
 12.5,
 0.0,
 0.0,
 0.0,
 7.14,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 25.0,


# trying to train the model on ISLRTC

In [194]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device = {device}")

device = cuda


In [226]:
def train(model, dataloader):
    model.train()
    running_loss = 0
    total = 0
    correct = 0
    l_preds = []
    l_labels = []
    metric = Metric('train')

    with tqdm(dataloader) as t:
        t.set_description('Training')
        for i, (features, labels) in enumerate(t):  
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            
            inp = features.permute(0,2,1)
            out = mstcn_trainable(inp, torch.ones(inp.size(), device=device))
            # print(out.shape)
            out_ll = out[-1]
            pred_prob = F.softmax(out_ll, dim=1)[0][1]

            # loss = criterion(out_ll[0][1], labels.squeeze().float())

            loss = 0
            batch_target = labels.squeeze()
            # mask = torch.ones_like(inp, dtype=torch.float)
            for ix, p in enumerate(out):
                if num_classes == 1:
                    loss += mse_red(p.transpose(2, 1).contiguous().view(-1, num_classes).squeeze(), batch_target.view(-1))
                else:
                    # print(p.shape)
                    # print(p.transpose(2, 1).contiguous().view(-1, num_classes).shape)
                    # print(batch_target.shape)
                    loss += ce(p.transpose(2, 1).contiguous().view(-1, num_classes), batch_target.view(-1))
                    loss += 0.15*torch.mean(torch.clamp(mse(F.log_softmax(p[:, :, 1:], dim=1), F.log_softmax(p.detach()[:, :, :-1], dim=1)), min=0, max=16))

            running_loss += loss.item()
            t.set_postfix({'loss':running_loss/(i+1)})
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            predicted = torch.tensor(np.where(np.asarray(pred_prob.cpu().detach().numpy()) > 0.5, 1, 0))

            total += batch_target.cpu().size(0)
            correct += (predicted == batch_target.cpu()).sum().item()
            
            metric.calc_scores_per_batch(predicted.to(device).unsqueeze(0), batch_target.unsqueeze(0),  batch_target.unsqueeze(0), None)
            
            l_preds.append(predicted.cpu().numpy())
            l_labels.append(batch_target.cpu().numpy())
    result = metric.calc_metrics()
    return 100*correct/total, running_loss/len(dataloader), result

In [234]:
def get_CP(features):
    model ='l2'
    pen = 80
    jump = 2
    
    if len(features) < 2:
        CP_dict[vid] = np.zeros(len(features))

    algo = rpt.Pelt(model=model, jump=jump).fit(features)
    res = algo.predict(pen=pen)
    res_np = [1 if ix in res else 0 for ix in range(len(features))]
    
    
    first_one_idx = None
    last_one_idx = None

    
    for i, elem in enumerate(res_np):
        if elem == 1:
            if first_one_idx is None:
                first_one_idx = i
            last_one_idx = i

    for i in range(len(res_np)):
         if i != first_one_idx and i != last_one_idx:
             res_np[i] = 0
            
    new_res_np = res_np.copy()
            
    for i in range(len(res_np)):
        if res_np[i] == 1:
            if i == 0:
                new_res_np[i:i+5] = [1, 1, 1, 1, 1]
            elif i == len(res_np)-1:
                new_res_np[i-5:i] = [1, 1, 1, 1, 1]
            else:
                new_res_np[i-2:i+2] = [1, 1, 1, 1, 1]
    
    return torch.LongTensor(new_res_np)

In [202]:
feat = torch.tensor(scipy.io.loadmat('/kaggle/input/testingdata/i3d_features/i3d_features/MVI_6104.MP4.mat')['features'], dtype=torch.float32)
print(feat.shape)

torch.Size([118, 1024])


In [223]:
with open('/kaggle/input/myislrtc/islrtc_CP_labels_dict.pkl', 'rb') as f:
    islrtc_cp_labels_dict = pickle.load(f)
    
print(len(islrtc_cp_labels_dict))

641


In [236]:
class Train_dataset(Dataset):
    
    def __init__(self, feature_path, label_path):
        super().__init__()
        self.feature_path = feature_path # path to .mat file
        self.label_path = label_path # key name to get the gt labels from the dict 
        
    def __len__(self):
        return 1
    
    def __getitem__(self, index):
        
        i3d_features_tensor = torch.tensor(scipy.io.loadmat(self.feature_path)['features'], dtype=torch.float32)
        gt_tensor = get_CP(i3d_features_tensor)
        
        len_gt = len(gt_tensor)
        len_i3d = len(i3d_features_tensor)
        
        if len_gt < len_i3d:
            i3d_features_tensor = i3d_features_tensor[:len_gt, :]
        elif len_gt >= len_i3d:
            gt_tensor = gt_tensor[:len_i3d]
        
        assert len(gt_tensor)==len(i3d_features_tensor)
        
        return i3d_features_tensor, gt_tensor

In [237]:
vid_names_list = list(islrtc_cp_labels_dict.keys())
my_path = '/kaggle/input/myislrtc/islrtc_i3d_features_bsl1k_bslcp/islrtc_i3d_features_bsl1k_bslcp'
feature_names_list = [os.path.join(my_path, str(x) + '_i3d.mat') for x in vid_names_list]

print(len(vid_names_list))
print(len(feature_names_list))

print(feature_names_list[0])
print(vid_names_list[0])

641
641
/kaggle/input/myislrtc/islrtc_i3d_features_bsl1k_bslcp/islrtc_i3d_features_bsl1k_bslcp/video78_i3d.mat
video78


In [238]:
new_model = torch.load('/kaggle/input/mstscnweights/mstcn_ISL_CMPL.model').to(device)

max_epochs = 5
for epoch in tqdm(range(max_epochs)):
    
    for i in range(len(vid_names_list)):
        dat = Train_dataset(feature_names_list[i], vid_names_list[i])
        loader = DataLoader(dat, batch_size=1)
        loss, acc, res = train(new_model, loader)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [245]:
# results of the trained model

mF1B_list = []
mF1S_list = []

for i in tqdm(range(len(mat_files_path))):
    my_dat = Test_dataset(mat_files_path[i], vid_files_path[i])
    my_loader = DataLoader(my_dat, batch_size=1)
    loss, acc, result = validate(new_model, my_loader)
    mF1B_list.append(result['mF1B'])
    mF1S_list.append(result['mF1S'])
    
print(len(mF1B_list), len(mF1S_list))

  0%|          | 0/176 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

176 176


In [246]:
statistics.mean(mF1S_list), statistics.mean(mF1B_list)

(33.38460227272727, 7.814034090909091)

# Random

In [ ]:
a = torch.load('/kaggle/input/mstscnweights/mstcn_ISL_CMPL.model')
i = 0
dat = Train_dataset(feature_names_list[i], vid_names_list[i])
loader = DataLoader(dat, batch_size=1)
for x, y in loader:
    print(x)
    print(y)
loss, acc, res = train(mstcn_trainable, loader)

In [124]:
i=17
my_dat = Test_dataset(mat_files_path[i], vid_files_path[i])
my_loader = DataLoader(my_dat, batch_size=1)

for x, y in my_loader:
    print(x.shape)
    print(y.shape)
    
loss, acc, result = validate(mstcn_trainable, my_loader)

result

torch.Size([1, 76, 1024])
torch.Size([1, 76])


  0%|          | 0/1 [00:00<?, ?it/s]

{'mF1B': 5.56,
 'F1S': 0.0,
 'mF1S': 0.0,
 'IoU': 9.814634146341463,
 'widthB': 4.428571428571429,
 'dist': 11.571428571428571,
 'detB': 7}

In [91]:
a = Test_dataset(Path('/kaggle/input/testingdata/i3d_features/i3d_features/MVI_6104.MP4.mat'), 'MVI_6104.MP4')

In [68]:
a = list(ground_truth_test['free.MP4'].values())
print(type(a))
print(len(a))
print(a[:5])
# print(a)

# b = a.type(torch.LongTensor)
# print(type(b))

<class 'list'>
94
[1, 1, 1, 1, 1]


In [71]:
testing_pairs = [] # will be a list of tuples (features_tensor, labels_tensor)
for vid_name in ground_truth_test.keys():
    
#     print(vid_name)
    features_path = os.path.join('/kaggle/input/testingdata/i3d_features/i3d_features', str(vid_name) + '.mat' )
#     print(features_path)
    i3d_features_tensor = torch.tensor(scipy.io.loadmat(features_path)['features'], dtype=torch.float32)
    gt = list(ground_truth_test[vid_name].values())
    
    num_i3d_frames = i3d_features_tensor.shape[0]
    num_gt_frames = len(gt)
    
    if num_i3d_frames < num_gt_frames:
        # truncate the gt
        gt = gt[:num_i3d_frames]
    elif num_i3d_frames >= num_gt_frames:
        # truncate the i3d frames
        i3d_features_tensor = i3d_features_tensor[:num_gt_frames, :]
        
    assert i3d_features_tensor.size(0) == len(gt)

#     print(len(gt))
#     print(i3d_features_tensor.shape)
    
    gt = torch.LongTensor(gt)
    testing_pairs.append((i3d_features_tensor, gt))
    
#     print('*' * 10)
#     print('\n\n')

print(len(testing_pairs))

176


In [73]:
testing_pairs[0][0].shape, testing_pairs[0][1].shape, testing_pairs[0][1].dtype

(torch.Size([86, 1024]), torch.Size([86]), torch.int64)

In [88]:
dat = TensorDataset(testing_pairs[0][0], testing_pairs[0][1])
my_loader = DataLoader(dat, batch_size=1)
for temp in my_loader:
    print(temp)
    print(temp[0].shape)
    break
    

[tensor([[0.0104, 0.0000, 0.0000,  ..., 0.0000, 0.0176, 0.0548]]), tensor([1])]
torch.Size([1, 1024])
